# Config

In [2]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import t
import numpy as np

# Problemas
## Exercício 05
### Item a

In [3]:
# Criando o DataFrame
data = {
    "Indivíduo": list(range(1, 21)),
    "Y": [96, 92, 106, 100, 98, 104, 110, 101, 116, 106,
          109, 100, 112, 105, 118, 108, 113, 112, 127, 117],
    "W": ["H", "M", "H", "M", "M", "H", "H", "M", "M", "H",
          "H", "M", "M", "M", "H", "H", "M", "M", "H", "H"],
    "X": [20, 20, 20, 20, 25, 25, 25, 25, 30, 30,
          30, 30, 35, 35, 35, 35, 40, 40, 40, 40],
    "Z": [90, 100, 80, 90, 100, 90, 80, 90, 70, 90,
          90, 80, 90, 80, 70, 90, 90, 90, 60, 80]
}

df = pd.DataFrame(data)

# Exibindo o DataFrame
df.head()

,Indivíduo,Y,W,X,Z
0,1,96,H,20,90
1,2,92,M,20,100
2,3,106,H,20,80
3,4,100,M,20,90
4,5,98,M,25,100


In [ ]:
# Ajustando o modelo de regressão Z = a + bX
model = smf.ols("Y ~ X", data=df).fit()
print(model.params)

Intercept    101.50
X             -0.55
dtype: float64


In [ ]:
# Gerando a tabela ANOVA
anova_table = sm.stats.anova_lm(model, typ=1)
print(anova_table)

            df  sum_sq  mean_sq         F    PR(>F)
X          1.0   302.5   302.50  3.408451  0.081375
Residual  18.0  1597.5    88.75       NaN       NaN


## Exercício 10

### Item a

In [41]:
intervalos = model.conf_int(alpha=0.05)

# Nomeando as colunas para facilitar a leitura
intervalos.columns = ["Limite Inferior", "Limite Superior"]

# Adicionando os nomes dos coeficientes
intervalos.index = ["Intercepto", "Coeficiente de X"]

intervalos

# Realizar validação manual...

,Limite Inferior,Limite Superior
Intercepto,82.208948,120.791052
Coeficiente de X,-1.175884,0.075884


In [51]:
# Valor da t-student com 18 graus de liberdade e 95% de confiança
t_critical = t.ppf(1 - 0.025, 18)

# -- Estimativa de sigma^2e
# Método 01: Usando a biblioteca
Se_1 = np.sqrt(model.mse_resid)
# Método 02: Usando a fórmula (SQRes / GLRes)
SQRes = model.ssr
GLRes = model.df_resid
Se_2 = np.sqrt(SQRes / GLRes)

print(Se_1 == Se_2)
Se = Se_1

# -- Raiz(1/Sxx)
Sxx = sum((df["X"] - df["X"].mean())**2)
raiz_Sxx = 1 / Sxx**(1/2)

# Estimativa do coeficiente de X
b_hat = model.params["X"]

# -- Intervalo de confiança
inferior = b_hat - t_critical * raiz_Sxx * Se
superior = b_hat + t_critical * raiz_Sxx * Se
print(inferior, superior)

b_hat, Sxx, Se

True
-1.1758843870487248 0.0758843870487258


(-0.5499999999999995, 1000.0, 9.420721840708387)

### Item c

In [56]:
-t_critical < (b_hat/Se) * (Sxx)**(1/2) < t_critical
# Tal que a estimativa não pertence a região crítica. Logo, a hipótese nula não é rejeitada.

True